# Creating a simple Presentation Module

This tutorial walks you through how to make your own **Presentation** module for the BCI.  By the end of this tutorial you will be able to:
 * design your own user interface screen with virtual 'buttons' selectable by brain signals
 * connect to the decoder and run through the calibration to train the BCI
 * use your designed user-interface screen to select buttons the on-screen buttons

Before running this tutorial you should have read [how an evoked bci works](https://mindaffect-bci.readthedocs.io/en/latest/how_an_evoked_bci_works.html) to get an overview of how this BCI works and it's main components, and run through [quickstart tutorial](quickstart.ipynb) to quickly test your installation and try the BCI.

* Note: The non-Jupyter Notebook version of this code and other presentation examples can be found in the* [examples/presentation directory](https://github.com/mindaffect/pymindaffectBCI/blob/doc/mindaffectBCI/examples/presentation/minimal_presentation.py)  

The presentation module is responsible for displaying the user-interface to the user with flickering options they can select from with brain signals (see [how an evoked bci works](https://mindaffect-bci.readthedocs.io/en/latest/how_an_evoked_bci_works.html) for more information).  In order for the presentation flickering to generate the strongest possible brain response, and hence maximise the BCI performance, it must display the correct stimuli to the user with precise timing and communicate this timing information to the MindAffect decoder.  Further, in order to train the decoding model presentation happens in two different modes: 

- _calibration_ mode where we cue the user where to attend to obtain correctly labelled brain data to train the machine learning algorithms in the decoder and
- _prediction_ mode where the user actually uses the BCI to make selections.

The *noisetag* module provides a number of tools to hide this complexity (different modes, timestamp recording) from the application developers.  Using the most extreme of these all the application developer has to do is provide a function to _draw_ the display as instructed by the noisetag module. To use it, we import the module and create the noisetag object:

In [1]:
from mindaffectBCI.noisetag import Noisetag, sumstats
nt = Noisetag()

## The Draw function
We will now write a function to draw the screen. Here we use the python gaming library [pyglet](www.pyglet.org) to draw 2 squares on the screen, with the given colors.


In [2]:
import pyglet
# define a simple 2-squares drawing function
def draw_squares(col1,col2):
    # draw square 1: @100,190 , width=100, height=100
    x=100; y=190; w=100; h=100;
    pyglet.graphics.draw(4,pyglet.gl.GL_QUADS,
                         ('v2f',(x,y,x+w,y,x+w,y+h,x,y+h)),
             ('c3f',(col1)*4))
    # draw square 2: @440,100
    x=640-100-100
    pyglet.graphics.draw(4,pyglet.gl.GL_QUADS,
                         ('v2f',(x,y,x+w,y,x+w,y+h,x,y+h)),
             ('c3f',(col2)*4))    


## Updating the Display
Now we write a function which, 1) asks the noisetag framework how the selectable squares should look, 2) updates the noisetag framework with information about how the display was updated.

In [3]:
# dictionary mapping from stimulus-state to colors
state2color={0:(.2,.2,.2), # off=grey
             1:(1,1,1),    # on=white
             2:(0,1,0),    # cue=green
             3:(0,0,1)}    # feedback=blue
def draw(dt):
    '''draw the display with colors from noisetag'''
    # send info on the *previous* stimulus state, with the recorded vsync time (if available)
    fliptime = window.lastfliptime if window.lastfliptime else nt.getTimeStamp()
    nt.sendStimulusState(timestamp=fliptime)
    # update and get the new stimulus state to display
    try : 
        nt.updateStimulusState()
        stimulus_state,target_state,objIDs,sendEvents=nt.getStimulusState()
    except StopIteration :
        pyglet.app.exit() # terminate app when noisetag is done
        return

    # draw the display with the instructed colors
    if stimulus_state : 
        draw_squares(state2color[stimulus_state[0]],
                     state2color[stimulus_state[1]])

    # some textual logging of what's happening
    if target_state is not None and target_state>=0:
        print("*" if target_state>0 else '.',end='',flush=True)
    else:
        print('.',end='',flush=True)

## Integrating Output

As a final step we integrate the behaviour of the output module created in the [Simple Output Tutorial](simple_output_tutorial.ipynb) by attaching a selection callback which will be called whenever a selection is made by the BCI. For now we will use the "Hello World" example created in the Output Tutorial and add a function that prints the object ID of our second square when selected.

In [4]:
def helloworld(objID):
    print("hello world")

def printID(objID):
    print("Selected: %d"%(objID))
    
# define a trivial selection handler    
def selectionHandler(objID):
    selection_mapping = {
        1:helloworld,
        2:printID
    }
    func = selection_mapping.get(objID)
    func(objID)

## Timing Accuracy 
Now, we need a bit of python hacking. Because our BCI depends on accurate timelock of the brain data (EEG) with the visual display, we need to have accurate time-stamps for when the display changes. Fortunately, pyglet allows us to get this accuracy as it provides a flip method on windows which blocks until the display is actually updated. Thus we can use this to generate accurate time-stamps. We do this by adding a time-stamp recording function to the windows normal flip method with the following magic:

In [5]:
import types

def timedflip(self):
    '''pseudo method type which records the timestamp for window flips'''
    type(self).flip(self) # call the 'real' flip method...
    self.lastfliptime=nt.getTimeStamp()    

Next, we initialize the window to display the stimulus, and setup the flip-time recording for it.  Be sure that you have [vsync](https://en.wikipedia.org/wiki/Screen_tearing#Vertical_synchronization) turned-on. Many graphics cards turn this off by default, as it (in theory) gives higher frame rates for gaming. However, for our system, frame-rate is less important than exact timing, hence always turn vsync on for visual Brain-Compuber-Interfaces!  

#### **Note:** always set ``fullscreen=True`` when using the presentation module to improve screen timing accuracy. We set it to ``False`` here so the tutorial stays visible when the stimulus is running.  

#### **Note:** When running in a notebook the pyglet window always starts minimized -- so if you can't see it check your task bar.

In [6]:
# Initialize the drawing window
# make a default window, with fixed size for simplicty, and vsync for timing
config = pyglet.gl.Config(double_buffer=True)
window = pyglet.window.Window(fullscreen=False, width=640,height=480, vsync=True, config=config)

# Setup the flip-time recording for this window
window.flip = types.MethodType(timedflip,window)
window.lastfliptime=None

## Start the BCI decoder in the background.
To successfully test your presentation module it is important to have the other components of the BCI running. As explained in the [quickstart tutorial](quickstart.ipynb), additionally to the presentation we build here, we need the Hub, Decoder, and Acquisition components for a functioning BCI.  
For a quick test (with fake data) of this presentation module you can run all these components with a given configuration file using.

N.B. if you run directly in this notebook, *don't forget to shutdown the decoder* at the end.

In [7]:
import mindaffectBCI.online_bci
config = mindaffectBCI.online_bci.load_config('fake_recogniser.json')
mindaffectBCI.online_bci.run(**config)

Loading config from: /home/christophe/Desktop/pymindaffectBCI/mindaffectBCI/fake_recogniser.json
Saving to /home/christophe/Desktop/pymindaffectBCI/mindaffectBCI/decoder/../../logs
Running command: ('java', '-jar', 'UtopiaServer.jar', '8400', '0', '/home/christophe/Desktop/pymindaffectBCI/mindaffectBCI/decoder/../../logs/mindaffectBCI_fake_recogniser.txt')
Server:cons
Server:open server ports
Server: Opening Port:8400
Server: Opening UDP Port:8400
Server: Utopia Server listening on: UDP:8400
wlp107s0->/fe80:0:0:0:2624:8734:7d78:985b%wlp107s0 ll=true
wlp107s0->/134.21.156.24 ll=false
Server: Opening SSDP-ipv4 Port:
wlp107s0->/fe80:0:0:0:2624:8734:7d78:985b%wlp107s0 ll=truesl=false
wlp107s0->/134.21.156.24 ll=falsesl=false
NIC=name:wlp107s0 (wlp107s0)
SSDP NIC: name:wlp107s0 (wlp107s0)
Server: Utopia Server listening on: SSDP:/0.0.0.0
Server: Opening SSDP-v6 Port:
wlp107s0->/fe80:0:0:0:2624:8734:7d78:985b%wlp107s0 ll=truesl=false
wlp107s0->/134.21.156.24 ll=falsesl=false
NIC=name:wlp107s

AttributeError: 'bool' object has no attribute 'lower'

DDDDH+
Dh+
DDDDHDh+
DDDDD.
Discovery returned 1 utopia-hub servers
Discovered utopia-hub on 134.21.156.24 ...
Tring to connect to: 134.21.156.24:-1
Trying to connect to: 134.21.156.24:8400
Connected!
geting some initial data to setup the ring buffer
Dh+
DD
Server: New Client Connection : /134.21.156.24:58746


Offset Update:0.0->-1.569447509E9
ClientInfoSubscribed /134.21.156.24:58746 to : [DEMSN]
HDDDDh+
DDH+
DDDDh+
DH1230 123 6.100951 201.607907 (samp,blk,s,hz)
DDDDDh+
DDDDDh+
HDDDDDDh+
H+
DDDDDHDh+
DDDDDDh+
DDDHDDh+
DDDDH+
DDh+
DDDHDDh+
DDEstimated sample rate 610 samp in 3.0 s =200.0
resample: 200.0->100.0hz rsrate=2.0
DDDDh+
DEstimated pre-processed sample rate=100.0
resample: 200.0->100.0hz rsrate=2.0
SigQ:
raw_power=([0.8033158 0.8130717 1.0349635 1.014711 ]/10)
pp_power=([0.18450783 0.02485542 0.0540537  0.19290168]/5)
noise2sig=[ 3.35383041 31.7120526  18.14695141  4.26024967]
SigQ:
raw_power=([0.96381366 0.96034175 1.0496273  0.97918236]/10)
pp_power=([0.45587001 0.47360897 0

noise2sig=[1.38815207 0.8627394  1.2513879  1.0654191 ]
.DH..D.D..D.DH..D.D..D.D..DD..DD..DD.D.D.D..D.D.DSigQ:
raw_power=([0.9725083 0.961594  1.0093905 0.9672774]/10)
pp_power=([0.46750113 0.47257754 0.4379556  0.44834973]/5)
noise2sig=[1.08022667 1.03478561 1.30477809 1.15741716]
H.D.D.D..D.D.H.D.D..D.D.
.D.D..D.D..D.D..D.D..D.D..D.DSigQ:
raw_power=([0.9602844 1.0009397 1.0419749 0.9985848]/10)
pp_power=([0.46789057 0.54223292 0.48494714 0.42816861]/5)
noise2sig=[1.05236967 0.84595896 1.14863606 1.33222328]
H..D.D..D.D..DHD..DD..DD.H.D.D7330 733 36.599895 200.273800 (samp,blk,s,hz).
D.D.D.D.D..D.D..DH.DSigQ:
raw_power=([0.9516037  0.97842336 0.9307389  1.0286763 ]/10)
pp_power=([0.39335421 0.46442234 0.44968047 0.47632733]/5)
noise2sig=[1.41920304 1.10675341 1.06977842 1.15959953]
.D.D..D.D..DH.D..D.D.
.D.D..D.D..D.D..D.D..D.D..D.DH..DSigQ:
raw_power=([1.0422322 1.0504357 0.9489363 1.0536747]/10)
pp_power=([0.49844583 0.50275036 0.50000206 0.53964531]/5)
noise2sig=[1.09096374 1.08937

noise2sig=[0.92876128 0.99481066 1.12057215 1.4819972 ]
..DD.D.D.D
.DH.D..D.D..D.DH..D.D..D.D..D.D..D12170 1217 60.800131 200.164043 (samp,blk,s,hz).
D..D.DSigQ:
raw_power=([0.98380244 0.9148431  0.98770136 1.0880634 ]/10)
pp_power=([0.52354001 0.40994989 0.45531363 0.47126834]/5)
noise2sig=[0.87913517 1.23159734 1.16927696 1.30879792]
..D.D..D.D..D.DH..D.D..D.D..DH.D..D.D..D.D..D.Dt.EHt.Etttt.EDtEEEDtt.EEEttt.tDSigQ:
raw_power=([0.98492444 0.98309726 0.9917235  1.3134922 ]/10)
pp_power=([0.50723893 0.48933736 0.48473435 0.41211594]/5)
noise2sig=[0.94173668 1.00903781 1.04591128 2.18719091]
EEEDE.Et.Et.Dtttt.EEEDE.Et.Et.D.
tttt.EEEDE.Et.tE.D.Htttt.EEEDtE.E.Et.D.tEttt.EEDEt.E.Et.D.tEttt.HEEDtE.E.Et.D.tE.ttt.EEtDE.E.tE.D.tE.ttt.tEEDEt.EEttttt.DEEEDEEH.Et.D.Et.tttt.EEDEE.Et.DSigQ:
raw_power=([1.0530177  0.99628854 1.0167146  1.8895006 ]/10)
pp_power=([0.528491   0.4732175  0.38373723 0.7376123 ]/5)
noise2sig=[0.9924989  1.10535014 1.64950728 1.56164468]
.tE.tttt.EEDEE.tE.D.tE.Etttt.EDEtE.

Fy=(1, 55, 256) Yest=2 Perr=0.21032500267028809
 Pred= P(80) PREDICTEDTARGETPROB -1 Yest=2 Perr=0.210325
.EDEEE.D.E.E.EExtract block: 14463693->14464787 = 1094ms
got: data 14463686.0->14464786.0 (111)  stimulus 14447642.0->14464778.0 (0.0>0)
Fy=(1, 66, 256) Yest=2 Perr=0.16890060901641846
 Pred= P(80) PREDICTEDTARGETPROB -1 Yest=2 Perr=0.168901
.DHEEEDE.EH.E.D.EExtract block: 14463693->14464936 = 1243ms
got: data 14463686.0->14464926.0 (125)  stimulus 14447642.0->14464926.0 (0.0>0)
Fy=(1, 80, 256) Yest=2 Perr=0.14122366905212402
 Pred= P(80) PREDICTEDTARGETPROB -1 Yest=2 Perr=0.141224
.EEDE.E.E.D.E.Extract block: 14463693->14465036 = 1343ms
got: data 14463686.0->14465026.0 (135)  stimulus 14447642.0->14465027.0 (0.0>0)
Fy=(1, 90, 256) Yest=2 Perr=0.27122020721435547
 Pred= P(80) PREDICTEDTARGETPROB -1 Yest=2 Perr=0.271220
.EEDEE.E.D.E.E.
Extract block: 14463693->14465136 = 1443ms
got: data 14463686.0->14465126.0 (145)  stimulus 14447642.0->14465127.0 (0.0>0)
Fy=(1, 100, 256) Yest=2 Per

noise2sig=[1.20701412 1.2810042  0.82739082 1.27914615]
.H

Alternatively you can run this config from the command line with:

```python3 -m mindaffectBCI.online_bci --config_file fake_recogniser.json``` 

Or from you Anaconda environment:   

```python -m mindaffectBCI.online_bci --config_file fake_recognizer.json```

**See our tutorial** [Running Custom Presentation](simple_integration_tutorial.ipynb) **to set-up a BCI using your own Presentation module**

## Run the Experiment!
To run the experiment we connect to the Hub, add our selection handler, tell the noisetag module to run a complete BCI 'experiment' with calibration and feedback mode, and start the pyglet main loop.

In [8]:
# Initialize the noise-tagging connection
nt.connect(timeout_ms=5000)
nt.addSelectionHandler(selectionHandler)

# tell the noisetag framework to run a full : calibrate->prediction sequence
nt.setnumActiveObjIDs(2)
nt.startExpt(nCal=4,nPred=10,duration=4)

# run the pyglet main loop
pyglet.clock.schedule(draw)
pyglet.app.run()

Trying to auto-discover the utopia-hub server
making discovery object
Using inteface: 134.21.156.24
SSDP::Query matched.  Sending response to/134.21.156.24:59568
Got location: 134.21.156.24D
.
DDDDh+
DDDDQH+
DDh+
DHDDDDDh+
HDDDD.
Dh+
DDDDDDh+
DQH+
DDDDHDh+
DDDHDDDh+
D.
DDDDDh+
DDDDH+
QDDh+
DDHDDDDh+
HDDD.
DDDh+
DDDDDDh+
H+
QDDDDDHDh+
DDHDDD.
Dh+
DDDDDDh+
DDDH+
QDDDh+
DDHDDDh+
HDD.
Discovery returned 1 utopia-hub servers
Discovered utopia-hub on 134.21.156.24 ...
Tring to connect to: 134.21.156.24:-1
Trying to connect to: 134.21.156.24:8400
Connected!

Server: New Client Connection : /134.21.156.24:42344

NewSubscriptions: PSNMEQ
ClientInfoSubscribed /134.21.156.24:42344 to : [PSNMEQ]

Offset Update:0.0->-1.569447508E9
HwaitFor: 120
.....D...D...D...Dh+
...D...D...D...D...Dh+
...DH+
...D.Q..D...D...D...Dh+
...DH...D...D..H.D...D...Dh+
...D...D...D...D...D...Dh+
...D...D...D...DH+
...D...Dh+
Q...D...D...D...DH...D...Dh+
.H.
.DStart Cal: 0/4 tgtidx=1M
tgtidx=1
.Hflicker: 60 frames, tgt 1


.EPF.Pred:P(80) PREDICTEDTARGETPROB -1569447510 Yest=2 Perr=0.168619E
D.E.E.ED.E.E.PFPred:P(80) PREDICTEDTARGETPROB -1569447510 Yest=2 Perr=0.130774
ED.E.E.ED.E.E.EDPF.Pred:P(80) PREDICTEDTARGETPROB -1569447510 Yest=2 Perr=0.121899E
.Eh+
.ED.E.E.ED.E.EPF.Pred:P(80) PREDICTEDTARGETPROB -1569447510 Yest=2 Perr=0.106766E
D.EH.E.ED.QE.E.PFEPred:P(80) PREDICTEDTARGETPROB -1569447510 Yest=2 Perr=0.129037
D.E.E.Eh+
D.E.E.EDHPF.Pred:P(80) PREDICTEDTARGETPROB -1569447510 Yest=2 Perr=0.115974E
.E.EH+
DH.E.E.ED.EPF.Pred:P(80) PREDICTEDTARGETPROB -1569447510 Yest=2 Perr=0.079514E
Selected: 2

3.feedback
S2295ms pred:1 sel:1  x
flicker: 60 frames, tgt 1
highlight: tgtidx=1 nframes=60
*D***Dh+
***D***D***D***D***D***Dh+
***D***D***D***D***D***Dh+
***D***D*Q*D****DH***DH+
***Hh+
D***

Start Pred: 1/10
tgtidx=-1
.D.NH2.stim, tgt:-1
flicker: 240 frames, tgt -1
 with selection
.ED.E.E.ED.E.E.ED.E.E.EDh+
.E.E.ED.E.E.ED.E.E.ED.E.E.ED.E.E.ED.Eh+
.E.ED.E.E.ED.E.E.ED.E.E.ED.EPF.Pred:P(80) PREDICTEDTARGETPROB

## Shutdown the decoder

In [10]:
import mindaffectBCI.online_bci
mindaffectBCI.online_bci.shutdown()